In [138]:
from ollama import chat
from ollama import ChatResponse

# Define the system prompt to guide the model's behavior
system_prompt = {
    'role': 'system',
    'content': (
        """You are a travel assistant. Your task is to determine if the user's prompt specifies:
        1. A starting city (sometimes denoted by "from").
        2. A destination city (sometimes denoted by "to").

        If either one is missing, explain nicely what information the user needs to input, 

        If both are present, respond only with 'VALID QUERY' and nothing else."""
    )
}

# Example user prompt (replace with actual user input)
# user_prompt = "Why is the sky blue?"
user_prompt = "I want to book a flight from chicago to newyork on january 1 2025"

# Call the chat model with both the system prompt and the user prompt
validation_response : ChatResponse = chat(model='llama3.2', messages=[
    system_prompt,
    {'role': 'user', 'content': user_prompt}],
    stream=False,
    options={"temperature":0.1}
)

validation_output = validation_response['message']['content']





# for chunk in validation_output:
#   print(chunk['message']['content'], end='', flush=True)

if 'VALID QUERY' in validation_output:
    # If input is valid, query the second LLM
    print('valid')
    # second_llm_output = query_second_llm(user_prompt)
    # print(second_llm_output)
else:
    # If input is invalid, respond with the validation output
    print(validation_output)


valid


In [155]:
import json

# Define the system prompt to guide the model's behavior
system_prompt = {
    'role': 'system',
    'content': (
        """
        You are a data parser for a travel related input.
        Your task is to read user input, and parse it into json format like below based on starting location, destination, and date.
        Only give the output as json format

        {"endpoint": "/shopping/flight-offers", "params": {"originLocationCode": "3 letter code", "destinationLocationCode": "3 letter code", "departureDate": "date format", "adults": 1}}
        """
    )
}

# Call the chat model with both the system prompt and the user prompt
json_response : ChatResponse = chat(model='llama3.2', messages=[
    system_prompt,
    {'role': 'user', 'content': user_prompt}],
    stream=False,
    options={"temperature":0.1}
)

json_response = json_response['message']['content']
response_json = json.loads(json_response)

In [ ]:
from amadeus import Client, ResponseError
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()


# API CALL TO AMADEUS
amadeus = Client(
    client_id=os.getenv('AMADEUS_CLIENT_ID'),
    client_secret=os.getenv('AMADEUS_CLIENT_SECRET')
)

try:
    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode=response_json['params']['originLocationCode'],
        destinationLocationCode=response_json['params']['destinationLocationCode'],
        departureDate=response_json['params']['departureDate'],
        adults=response_json['params']['adults'])
    
    rd = response.data
except ResponseError as error:
    print(error)

In [159]:
# Function to calculate the number of stops
def get_num_stops(offer):
    return len(offer['itineraries'][0]['segments']) - 1



# Function to filter by stops and cabin class
def filter_by_stops_and_cabin(data, stops, cabin_class, top_n):
    # Filter by number of stops
    filtered_by_stops = [offer for offer in data if get_num_stops(offer) == stops]
    
    # Filter by cabin class
    filtered_by_cabin = [
        offer for offer in filtered_by_stops 
        if all(fare['cabin'] == cabin_class for fare in offer['travelerPricings'][0]['fareDetailsBySegment'])
    ]
    
    # Sort by price and return the top `n` offers
    return sorted(filtered_by_cabin, key=lambda x: float(x['price']['total']))[:top_n]

n = 2  # Number of top choices to return

# Get results for 0-stop flights in economy and business
zero_stops_economy = filter_by_stops_and_cabin(rd, stops=0, cabin_class="ECONOMY", top_n=n)
zero_stops_business = filter_by_stops_and_cabin(rd, stops=0, cabin_class="PREMIUM_ECONOMY", top_n=n)

# Get results for 1-stop flights in economy and business
one_stop_economy = filter_by_stops_and_cabin(rd, stops=1, cabin_class="ECONOMY", top_n=n)
one_stop_business = filter_by_stops_and_cabin(rd, stops=1, cabin_class="PREMIUM_ECONOMY", top_n=n)


In [167]:
from datetime import datetime



def feature_engineering(data, keys_to_extract):
    """
    Adds a 'flightinfo' key and a 'layover_minutes' key to each item in the dataset.
    - 'flightinfo': Merges details from segments and fareDetailsBySegment.
    - 'layover_minutes': Calculates the layover time between the first arrival and second departure.

    Parameters:
    - data (list): A list of dictionaries, where each dictionary contains 'itineraries' 
                   and 'travelerPricings' keys.

    Returns:
    - list: The updated dataset with 'flightinfo' and 'layover_minutes' added to each item.
    """
    for item in data:
        # Extract segments and fare details
        segments = item['itineraries'][0]['segments']
        fare_details = item['travelerPricings'][0]['fareDetailsBySegment']

        # Merge flightinfo
        merged_flight_info = []
        for segment in segments:
            for fare in fare_details:
                if segment['id'] == fare['segmentId']:
                    merged_flight_info.append({
                        'departure': segment['departure'],
                        'arrival': segment['arrival'],
                        'carrierCode': segment['carrierCode'],
                        'duration': segment['duration'],
                        'cabin': fare['cabin'],
                        # 'includedCheckedBags': fare['includedCheckedBags']
                    })
        item['flightinfo'] = merged_flight_info

        # Calculate layover_minutes
        if len(segments) > 1:  # Ensure there are at least two segments to calculate layover
            first_arrival = segments[0]['arrival']['at']
            second_departure = segments[1]['departure']['at']

            # Convert times to datetime objects
            first_arrival_time = datetime.fromisoformat(first_arrival)
            second_departure_time = datetime.fromisoformat(second_departure)

            # Calculate the time difference in minutes
            time_difference_minutes = (second_departure_time - first_arrival_time).total_seconds() / 60
            item['layover_minutes'] = time_difference_minutes
        else:
            item['layover_minutes'] = 0

    data = [{key: item[key] for key in keys_to_extract if key in item} for item in data]

    return data

keys_to_extract = ['numberOfBookableSeats','flightinfo','layover_minutes','price']


ze = feature_engineering(zero_stops_economy, keys_to_extract)
zb = feature_engineering(zero_stops_business, keys_to_extract)
oe = feature_engineering(one_stop_economy, keys_to_extract)
ob = feature_engineering(one_stop_business, keys_to_extract)

In [172]:
from ollama import chat
from ollama import ChatResponse

# Define the system prompt to guide the model's behavior
system_prompt = {
    'role': 'system',
    'content': (
        """
        You are a travel assistant.
        The input will be a json format input with flight information
        Given the information, summarize the choices the user has regarding the information.
        Speak as if you are the tour guide ans speaking to a tourist.
        
        """
    )
}

# Example user prompt (replace with actual user input)
# user_prompt = "Why is the sky blue?"
user_prompt = str(ze)

# Call the chat model with both the system prompt and the user prompt
response : ChatResponse = chat(model='llama3.2', messages=[
    system_prompt,
    {'role': 'user', 'content': user_prompt}],
    stream=True,
    options={"temperature":0.1}
)

for chunk in response:
  print(chunk['message']['content'], end='', flush=True)

Welcome, traveler! I'm your tour guide today. We're going to explore the fascinating world of flight information.

As we examine this JSON data, you'll notice that there are several key aspects of your flight that we can discuss. Let's break them down:

1. **Flight Details**: You have a total of 7 bookable seats on this flight. The departure airport is ORD (O'Hare International Airport), and the arrival airport is JFK (John F. Kennedy International Airport). The flight duration is approximately 2 hours and 9 minutes.

2. **Layover Information**: We don't have any layovers scheduled for you, as the layover time is just 0 minutes. This means you'll be straight from departure to arrival without any breaks in between.

3. **Flight Price**: The total price of your ticket is €183.12 (Euros). This includes a base fare of €157.00 and some additional fees:
	* A checked bag fee of €37.95.
	* No other fees are applicable, as the supplier has waived them.

Now that we've gone over these key aspect

In [1]:
from ollama import chat
from ollama import ChatResponse

# Define the system prompt to guide the model's behavior
system_prompt = {
    'role': 'system',
    'content': (
        """
        you are a helpful assistant
        """
    )
}

# Example user prompt (replace with actual user input)
# user_prompt = "Why is the sky blue?"
user_prompt = "What is the longitude and latitude of paris"

# Call the chat model with both the system prompt and the user prompt
response : ChatResponse = chat(model='llama3.2', messages=[
    system_prompt,
    {'role': 'user', 'content': user_prompt}],
    stream=True,
    options={"temperature":0.1}
)

for chunk in response:
  print(chunk['message']['content'], end='', flush=True)

The longitude and latitude of Paris, France are:

* Latitude: 48.8567° N
* Longitude: 2.3522° E

Note that these coordinates refer to the center of Paris, which is located at the intersection of the Seine River and the Boulevard Saint-Michel.

In [1]:
from amadeus import Client, ResponseError
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()


# API CALL TO AMADEUS
amadeus = Client(
    client_id=os.getenv('AMADEUS_CLIENT_ID'),
    client_secret=os.getenv('AMADEUS_CLIENT_SECRET')
)

try:
    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode=response_json['params']['originLocationCode'],
        destinationLocationCode=response_json['params']['destinationLocationCode'],
        departureDate=response_json['params']['departureDate'],
        adults=response_json['params']['adults'])
    
    rd = response.data
except ResponseError as error:
    print(error)

NameError: name 'response_json' is not defined

In [18]:
# Get list of hotels by city code
hotels = amadeus.reference_data.locations.hotels.by_city.get(cityCode='PAR', ratings=[5], radius=30)

In [19]:
len(hotels.data)

107

In [20]:
hotels.data

[{'chainCode': 'HN',
  'iataCode': 'PAR',
  'dupeId': 502345065,
  'name': 'TEST PROPERTY FOR API ACTIVATE/DE-ACTIVA',
  'hotelId': 'HNPARNUJ',
  'geoCode': {'latitude': 48.85315, 'longitude': 2.34513},
  'address': {'countryCode': 'FR'},
  'distance': {'value': 0.27, 'unit': 'KM'},
  'rating': 5,
  'lastUpdate': '2024-09-02T10:57:00'},
 {'chainCode': 'BW',
  'iataCode': 'PAR',
  'dupeId': 700054608,
  'name': 'BW PREMIER ROYAL SAINT MICHEL',
  'hotelId': 'BWPAR599',
  'geoCode': {'latitude': 48.85269, 'longitude': 2.34384},
  'address': {'countryCode': 'FR'},
  'distance': {'value': 0.37, 'unit': 'KM'},
  'rating': 5,
  'lastUpdate': '2023-06-15T09:58:55'},
 {'chainCode': 'LX',
  'iataCode': 'PAR',
  'dupeId': 700011706,
  'name': 'HOTEL RELAIS CHRISTINE',
  'hotelId': 'LXPARCHP',
  'geoCode': {'latitude': 48.85441, 'longitude': 2.33994},
  'address': {'countryCode': 'FR'},
  'distance': {'value': 0.66, 'unit': 'KM'},
  'rating': 5,
  'lastUpdate': '2023-06-15T09:54:45'},
 {'chainCode

In [37]:
poi = amadeus.reference_data.locations.get(keyword='PARIS', subType='CITY')
# points_of_interest.get(params='PAR')

In [38]:
poi.data

[{'type': 'location',
  'subType': 'CITY',
  'name': 'PARIS',
  'detailedName': 'PARIS/FR',
  'id': 'CPAR',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/CPAR',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'PAR',
  'geoCode': {'latitude': 48.85334, 'longitude': 2.34889},
  'address': {'cityName': 'PARIS',
   'cityCode': 'PAR',
   'countryName': 'FRANCE',
   'countryCode': 'FR',
   'regionCode': 'EUROP'},
  'analytics': {'travelers': {'score': 68}}},
 {'type': 'location',
  'subType': 'CITY',
  'name': 'LE TOUQUET PARIS PLAGE',
  'detailedName': 'LE TOUQUET PARIS PLAGE/FR',
  'id': 'CLTQ',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/CLTQ',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'LTQ',
  'geoCode': {'latitude': 50.5175, 'longitude': 1.62056},
  'address': {'cityName': 'LE TOUQUET PARIS PLAGE',
   'cityCode': 'LTQ',
   'countryName': 'FRANCE',
   'countryCode': 'FR',
   'reg

In [ ]:
activity = amadeus.shopping.activities.get(latitude=40.41436995, longitude=-3.69170868, radius = 20)

In [58]:
filtered_data = [
    entry for entry in activity.data
    if 'description' in entry and len(entry['description'].split()) >= 200
]

len(filtered_data)


319

In [56]:
filtered_data

[{'type': 'activity',
  'id': '6443720',
  'self': {'href': 'https://test.api.amadeus.com/v1/shopping/activities/6443720',
   'methods': ['GET']},
  'name': 'Taylor Swift',
  'shortDescription': 'American pop star Taylor Swift brings The Eras Tour to Spain, a unique concert experience celebrating her entire musical career at Santiago Bernabéu Stadium on May 29 and 30, 2024.',
  'description': "The current American pop star Taylor Swift offers a double date of concerts in Spain as part of his successful tour The Eras Tour in which he reviews all his musical career. The Santiago Bernabéu Stadium will be in charge of hosting this expected event on May 29 and 30, 2024.\r\n\r\nTaylor Swift’s new tour, which began in the United States in March 2023, has broken attendance records for each concert. In 2024, The Eras Tour will live its European stage stopping in Madrid where attendees can enjoy a show specially designed to celebrate all the stages of their musical career. Each stage of the conc

In [6]:
from ollama import chat
from ollama import ChatResponse
import time
import json
from amadeus import Client, ResponseError
from datetime import datetime
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()


def stream_output(output, delay=0.05):
    for char in output.split():
        print(char, end=' ', flush=True)
        time.sleep(delay)


# Function to calculate the number of stops
def get_num_stops(offer):
    return len(offer['itineraries'][0]['segments']) - 1



# Function to filter by stops and cabin class
def filter_by_stops_and_cabin(data, stops, cabin_class, top_n):
    # Filter by number of stops
    filtered_by_stops = [offer for offer in data if get_num_stops(offer) == stops]
    
    # Filter by cabin class
    filtered_by_cabin = [
        offer for offer in filtered_by_stops 
        if all(fare['cabin'] == cabin_class for fare in offer['travelerPricings'][0]['fareDetailsBySegment'])
    ]
    
    # Sort by price and return the top `n` offers
    return sorted(filtered_by_cabin, key=lambda x: float(x['price']['total']))[:top_n]

def feature_engineering(data, keys_to_extract):
    """
    Adds a 'flightinfo' key and a 'layover_minutes' key to each item in the dataset.
    - 'flightinfo': Merges details from segments and fareDetailsBySegment.
    - 'layover_minutes': Calculates the layover time between the first arrival and second departure.

    Parameters:
    - data (list): A list of dictionaries, where each dictionary contains 'itineraries' 
                and 'travelerPricings' keys.

    Returns:
    - list: The updated dataset with 'flightinfo' and 'layover_minutes' added to each item.
    """
    for item in data:
        # Extract segments and fare details
        segments = item['itineraries'][0]['segments']
        fare_details = item['travelerPricings'][0]['fareDetailsBySegment']

        # Merge flightinfo
        merged_flight_info = []
        for segment in segments:
            for fare in fare_details:
                if segment['id'] == fare['segmentId']:
                    merged_flight_info.append({
                        'departure': segment['departure'],
                        'arrival': segment['arrival'],
                        'carrierCode': segment['carrierCode'],
                        'duration': segment['duration'],
                        'cabin': fare['cabin'],
                        # 'includedCheckedBags': fare['includedCheckedBags']
                    })
        item['flightinfo'] = merged_flight_info

        # Calculate layover_minutes
        if len(segments) > 1:  # Ensure there are at least two segments to calculate layover
            first_arrival = segments[0]['arrival']['at']
            second_departure = segments[1]['departure']['at']

            # Convert times to datetime objects
            first_arrival_time = datetime.fromisoformat(first_arrival)
            second_departure_time = datetime.fromisoformat(second_departure)

            # Calculate the time difference in minutes
            time_difference_minutes = (second_departure_time - first_arrival_time).total_seconds() / 60
            item['layover_minutes'] = time_difference_minutes
        else:
            item['layover_minutes'] = 0

    data = [{key: item[key] for key in keys_to_extract if key in item} for item in data]

    return data




def NomadAI(prompt):

    # Define the system prompt to guide the model's behavior
    system_prompt = {
        'role': 'system',
        'content': (
            """You are a travel assistant. Your task is to determine if the user's prompt specifies:
            1. A starting city (sometimes denoted by "from").
            2. A destination city (sometimes denoted by "to").

            If either one is missing, explain nicely what information the user needs to input, 

            If both are present, respond only with 'VALID QUERY' and nothing else."""
        )
    }

    # Example user prompt (replace with actual user input)
    # user_prompt = "Why is the sky blue?"
    # user_prompt = "I want to book a flight from chicago to newyork on january 1 2025"

    # Call the chat model with both the system prompt and the user prompt
    validation_response : ChatResponse = chat(model='llama3.2', messages=[
        system_prompt,
        {'role': 'user', 'content': prompt}],
        stream=False,
        options={"temperature":0.1}
    )

    validation_output = validation_response['message']['content']

    # for chunk in validation_output:
    #   print(chunk['message']['content'], end='', flush=True)

    if 'VALID QUERY' in validation_output:
        # If input is valid, query the second LLM
        pass
        # second_llm_output = query_second_llm(user_prompt)
        # print(second_llm_output)
    else:
        # If input is invalid, respond with the validation output
        return stream_output(validation_output)



    # FINE TUNE MOEL GOES HERE
    # Define the system prompt to guide the model's behavior
    system_prompt = {
        'role': 'system',
        'content': (
            """
            You are a data parser for a travel related input.
            Your task is to read user input, and parse it into json format like below based on starting location, destination, and date.
            Only give the output as json format

            {"endpoint": "/shopping/flight-offers", "params": {"originLocationCode": "3 letter code", "destinationLocationCode": "3 letter code", "departureDate": "date format", "adults": 1}}
            """
        )
    }

    # Call the chat model with both the system prompt and the user prompt
    json_response : ChatResponse = chat(model='llama3.2', messages=[
        system_prompt,
        {'role': 'user', 'content': prompt}],
        stream=False,
        options={"temperature":0.1}
    )

    json_response = json_response['message']['content']
    response_json = json.loads(json_response)



    
    # API CALL TO AMADEUS
    amadeus = Client(
        client_id=os.getenv('AMADEUS_CLIENT_ID'),
        client_secret=os.getenv('AMADEUS_CLIENT_SECRET')
    )

    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=response_json['params']['originLocationCode'],
            destinationLocationCode=response_json['params']['destinationLocationCode'],
            departureDate=response_json['params']['departureDate'],
            adults=response_json['params']['adults'])
        
        rd = response.data
    except ResponseError as error:
        print(error)

    n = 2  # Number of top choices to return

    # Get results for 0-stop flights in economy and business
    zero_stops_economy = filter_by_stops_and_cabin(rd, stops=0, cabin_class="ECONOMY", top_n=n)
    zero_stops_business = filter_by_stops_and_cabin(rd, stops=0, cabin_class="PREMIUM_ECONOMY", top_n=n)

    # Get results for 1-stop flights in economy and business
    one_stop_economy = filter_by_stops_and_cabin(rd, stops=1, cabin_class="ECONOMY", top_n=n)
    one_stop_business = filter_by_stops_and_cabin(rd, stops=1, cabin_class="PREMIUM_ECONOMY", top_n=n)

    keys_to_extract = ['numberOfBookableSeats','flightinfo','layover_minutes','price']

    ze = feature_engineering(zero_stops_economy, keys_to_extract)
    zb = feature_engineering(zero_stops_business, keys_to_extract)
    oe = feature_engineering(one_stop_economy, keys_to_extract)
    ob = feature_engineering(one_stop_business, keys_to_extract)




    # JSON TO HUMAN OUTPUT
    # Define the system prompt to guide the model's behavior
    system_prompt = {
        'role': 'system',
        'content': (
            """
            You are a travel assistant.
            The input will be a json format input with flight information
            Given the information, summarize the choices the user has regarding the information.
            Speak as if you are explaining the details for a customer.
            
            """
        )
    }

    # Example user prompt (replace with actual user input)
    # user_prompt = "Why is the sky blue?"
    user_prompt = str(ze)

    # Call the chat model with both the system prompt and the user prompt
    response : ChatResponse = chat(model='llama3.2', messages=[
        system_prompt,
        {'role': 'user', 'content': user_prompt}],
        stream=True,
        options={"temperature":0.1}
    )

    for chunk in response:
        print(chunk['message']['content'], end='', flush=True)

In [7]:
NomadAI("I want to book a flight from chicago to newyork on january 1 2025")

Welcome to our travel assistance! I'd be happy to help you understand the details of your flight.

Based on the information provided, here are the choices you have regarding your flight:

1. **Flight Details**: You can see that your flight is departing from Chicago's O'Hare International Airport (ORD) and arriving at John F. Kennedy International Airport (JFK). The flight duration is approximately 2 hours and 9 minutes.

2. **Seat Availability**: There are 7 bookable seats available on this flight, which means you can choose one of these seats when booking your ticket.

3. **Layover Time**: You don't have any layovers scheduled for this flight, as the total travel time is just under 3 hours.

4. **Flight Carrier**: The airline operating this flight is JetBlue (B6).

5. **Additional Services**: There's an additional service charge of €37.95 for checked bags on your flight.

6. **Price Breakdown**: Your total fare includes:
	* Base price: €157.00
	* Fees: €0.00 (no additional fees)
	* Ad

In [5]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ollama import chat
from ollama import ChatResponse

def query_refiner(log, prompt):
    # Define the system prompt to guide the model's behavior
    system_prompt = {
        'role': 'system',
        'content': (
            """
            Given the following user query and conversation log, formulate a question that would be the most relevant to provide the user with an answer from a knowledge base.
            Just output the new question
            """
        )
    }

    # Call the chat model with both the system prompt and the user prompt
    response : ChatResponse = chat(model='llama3.2', messages=[
        system_prompt,
        {'role': 'user', 'content': f"Log : {log} / Prompt : {prompt}"}],
        stream=False,
        options={"temperature":0.1}
    )

    return response['message']['content']

In [4]:
print(query_refiner({'user': 'How is the economomy in india', 'assistant':'Its doing great'}, "What about the relationship with the united states"))

What is India's economic relationship with the United States?


In [ ]:
def query_refiner(conversation, query):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Given the following user query and conversation log, formulate a question that would be the most relevant to provide the user with an answer from a knowledge base.\n\nCONVERSATION LOG: \n{conversation}\n\nQuery: {query}\n\nRefined Query:",
        temperature=0.7,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response['choices'][0]['text'].strip()